In [1]:
!pip install wandb 


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Conv2D,Add,MaxPooling2D, Dense, BatchNormalization,Input,Flatten, Dropout,GlobalMaxPooling2D,Lambda

from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

import wandb

2023-02-12 07:48:10.883851: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-12 07:48:11.010171: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-12 07:48:11.053931: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a suppor

In [3]:
from pathlib import Path

PATH = Path('./CUB_200_2011')
labels = pd.read_csv(PATH/"image_class_labels.txt", header=None, sep=" ")
labels.columns = ["id", "label"]

train_test = pd.read_csv(PATH/"train_test_split.txt", header=None, sep=" ")
train_test.columns = ["id", "is_train"]

images = pd.read_csv(PATH/"images.txt", header=None, sep=" ")
images.columns = ["id", "bird_name"]

classes = pd.read_csv(PATH/"classes.txt", header=None, sep=" ")
classes.columns = ["id", "class"]

df_1 = pd.merge(images, labels, on='id')
df = pd.merge(df_1, train_test, on='id')

train, test = [], []
train_test = df.iloc[:,-1]

for i in range(len(df)):
    if train_test[i] == 1:
        train.append(df.loc[i])
    else:
        test.append(df.loc[i])
        
train = pd.DataFrame(train)
test = pd.DataFrame(test)

In [4]:
def preprocessing(image):
    resized_image = tf.image.resize(image, [112,112])
    return tf.keras.applications.vgg16.preprocess_input(resized_image)

X_train, y_train = [], []
path = ''

for i, row in train.iterrows():
    bird_name = row['bird_name']
    label = row['label']
    path = f'./CUB_200_2011/images/{bird_name}'
    img = plt.imread(path)
    
    if len(img.shape) == 3 or len(img.shape) == 4:
        img = preprocessing(img)
        X_train.append(img)
        y_train.append(label)
        
X_train = np.array(X_train)
y_train = np.array(y_train)

y_train = tf.keras.utils.to_categorical(y_train)



X_test, y_test = [], []
path = ''

for id, row in test.iterrows():
    bird_name = row['bird_name']
    label = row['label']
    path = f'./CUB_200_2011/images/{bird_name}'
    img = plt.imread(path)
    
    if len(img.shape) == 3 or len(img.shape) == 4:
        img = preprocessing(img)
        X_test.append(img)
        y_test.append(label)
X_test = np.array(X_test)
y_test = np.array(y_test)       
y_test = tf.keras.utils.to_categorical (y_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

2023-02-12 07:48:14.288492: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-12 07:48:14.928023: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22277 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6


(5990, 112, 112, 3)
(5990, 201)
(5790, 112, 112, 3)
(5790, 201)


In [5]:
from sklearn.model_selection import train_test_split

X_tn, X_val, y_tn, y_val = train_test_split(X_train, y_train, test_size=0.14, random_state=42)

In [6]:
print(X_tn.shape)
print(y_tn.shape)

print(X_val.shape)
print(y_val.shape)

(5151, 112, 112, 3)
(5151, 201)
(839, 112, 112, 3)
(839, 201)


In [7]:
wandb.init(project="Bird_Pretrain", entity="hcim", name='EfficientNetV2L')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense, Activation, MaxPool2D, BatchNormalization, Dropout
from tensorflow.keras import regularizers

eff = tf.keras.applications.EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(112,112,3), pooling='avg')

x = Flatten()(eff.output)

x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(201, activation='softmax')(x)

model = Model(eff.input, output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 112, 112, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 112, 112, 3)  0           ['input_2[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 56, 56, 32)   864         ['rescaling_1[0][0]']            
                                                                                                  
 stem_bn (BatchNormalization)   (None, 56, 56, 32)   128         ['stem_conv[0][0]']          

In [10]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=15, horizontal_flip=True, vertical_flip=True)
datagen.fit(X_tn)

In [11]:
from tensorflow.keras import callbacks
class WandbCallback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        wandb.log(logs)

In [14]:
import tensorflow_addons.metrics
optimizer = tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9, decay=0.0005)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[tf.keras.metrics.CategoricalAccuracy(),tensorflow_addons.metrics.F1Score(num_classes=201, average='macro')]) 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [15]:
hist = model.fit_generator(datagen.flow(X_tn, y_tn, batch_size=32), epochs=800, validation_data=(X_val, y_val), callbacks=[WandbCallback()])

Epoch 1/800


/tmp/ipykernel_363/40923217.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(datagen.flow(X_tn, y_tn, batch_size=32), epochs=800, validation_data=(X_val, y_val), callbacks=[WandbCallback()])
2023-02-12 07:58:16.855848: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-12 07:58:18.185561: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


161/161 [==============================] - 61s 187ms/step - loss: 5.3502 - categorical_accuracy: 0.0043 - f1_score: 0.0032 - val_loss: 5.3102 - val_categorical_accuracy: 0.0060 - val_f1_score: 0.0024
Epoch 2/800
161/161 [==============================] - 24s 151ms/step - loss: 5.3207 - categorical_accuracy: 0.0047 - f1_score: 0.0038 - val_loss: 5.3041 - val_categorical_accuracy: 0.0060 - val_f1_score: 0.0032
Epoch 3/800
161/161 [==============================] - 24s 151ms/step - loss: 5.3089 - categorical_accuracy: 0.0050 - f1_score: 0.0038 - val_loss: 5.3021 - val_categorical_accuracy: 0.0095 - val_f1_score: 0.0063
Epoch 4/800
161/161 [==============================] - 25s 152ms/step - loss: 5.3086 - categorical_accuracy: 0.0058 - f1_score: 0.0047 - val_loss: 5.3009 - val_categorical_accuracy: 0.0131 - val_f1_score: 0.0077
Epoch 5/800
161/161 [==============================] - 24s 151ms/step - loss: 5.3077 - categorical_accuracy: 0.0070 - f1_score: 0.0054 - val_loss: 5.3033 - val_cate

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



161/161 [==============================] - 24s 151ms/step - loss: 3.4560 - categorical_accuracy: 0.1881 - f1_score: 0.1686 - val_loss: 3.3601 - val_categorical_accuracy: 0.2133 - val_f1_score: 0.1723
Epoch 161/800
161/161 [==============================] - 25s 156ms/step - loss: 3.4626 - categorical_accuracy: 0.1875 - f1_score: 0.1675 - val_loss: 3.3132 - val_categorical_accuracy: 0.2300 - val_f1_score: 0.1859
Epoch 162/800
161/161 [==============================] - 25s 154ms/step - loss: 3.4259 - categorical_accuracy: 0.1893 - f1_score: 0.1709 - val_loss: 3.3064 - val_categorical_accuracy: 0.2312 - val_f1_score: 0.1818
Epoch 163/800
161/161 [==============================] - 24s 151ms/step - loss: 3.4332 - categorical_accuracy: 0.1945 - f1_score: 0.1760 - val_loss: 3.3027 - val_categorical_accuracy: 0.2288 - val_f1_score: 0.1830
Epoch 164/800
161/161 [==============================] - 24s 150ms/step - loss: 3.4252 - categorical_accuracy: 0.1996 - f1_score: 0.1801 - val_loss: 3.2916 - 

...

161/161 [==============================] - 25s 152ms/step - loss: 1.1995 - categorical_accuracy: 0.6585 - f1_score: 0.6514 - val_loss: 2.0011 - val_categorical_accuracy: 0.4839 - val_f1_score: 0.4480
Epoch 680/800
161/161 [==============================] - 24s 152ms/step - loss: 1.1848 - categorical_accuracy: 0.6610 - f1_score: 0.6516 - val_loss: 2.0145 - val_categorical_accuracy: 0.4791 - val_f1_score: 0.4415
Epoch 681/800
161/161 [==============================] - 24s 149ms/step - loss: 1.1905 - categorical_accuracy: 0.6572 - f1_score: 0.6495 - val_loss: 1.9966 - val_categorical_accuracy: 0.4815 - val_f1_score: 0.4479
Epoch 682/800
161/161 [==============================] - 24s 151ms/step - loss: 1.1712 - categorical_accuracy: 0.6595 - f1_score: 0.6522 - val_loss: 1.9928 - val_categorical_accuracy: 0.4863 - val_f1_score: 0.4485
Epoch 688/800
161/161 [==============================] - 24s 151ms/step - loss: 1.2023 - categorical_accuracy: 0.6599 - f1_score: 0.6513 - val_loss: 2.0056 - 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



161/161 [==============================] - 25s 152ms/step - loss: 1.1160 - categorical_accuracy: 0.6762 - f1_score: 0.6686 - val_loss: 2.0015 - val_categorical_accuracy: 0.4899 - val_f1_score: 0.4562
Epoch 726/800
161/161 [==============================] - 25s 152ms/step - loss: 1.0984 - categorical_accuracy: 0.6758 - f1_score: 0.6680 - val_loss: 1.9880 - val_categorical_accuracy: 0.4875 - val_f1_score: 0.4522
Epoch 727/800
161/161 [==============================] - 24s 151ms/step - loss: 1.1332 - categorical_accuracy: 0.6647 - f1_score: 0.6560 - val_loss: 1.9861 - val_categorical_accuracy: 0.4875 - val_f1_score: 0.4487
Epoch 728/800
161/161 [==============================] - 24s 151ms/step - loss: 1.1220 - categorical_accuracy: 0.6731 - f1_score: 0.6646 - val_loss: 1.9903 - val_categorical_accuracy: 0.4923 - val_f1_score: 0.4572
Epoch 729/800
161/161 [==============================] - 24s 151ms/step - loss: 1.1209 - categorical_accuracy: 0.6799 - f1_score: 0.6723 - val_loss: 1.9995 - 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



161/161 [==============================] - 24s 150ms/step - loss: 1.0721 - categorical_accuracy: 0.6925 - f1_score: 0.6848 - val_loss: 2.0125 - val_categorical_accuracy: 0.4827 - val_f1_score: 0.4482
Epoch 780/800
161/161 [==============================] - 24s 151ms/step - loss: 1.0656 - categorical_accuracy: 0.6913 - f1_score: 0.6845 - val_loss: 1.9891 - val_categorical_accuracy: 0.4803 - val_f1_score: 0.4466
Epoch 781/800
161/161 [==============================] - 24s 151ms/step - loss: 1.0401 - categorical_accuracy: 0.6952 - f1_score: 0.6890 - val_loss: 1.9818 - val_categorical_accuracy: 0.4863 - val_f1_score: 0.4513
Epoch 782/800
161/161 [==============================] - 25s 152ms/step - loss: 1.0843 - categorical_accuracy: 0.6828 - f1_score: 0.6769 - val_loss: 1.9889 - val_categorical_accuracy: 0.4851 - val_f1_score: 0.4499
Epoch 783/800
161/161 [==============================] - 24s 150ms/step - loss: 1.0400 - categorical_accuracy: 0.6981 - f1_score: 0.6923 - val_loss: 2.0018 - 

In [16]:
print(model.evaluate(X_train, y_train))
print(model.evaluate(X_val, y_val))
print(model.evaluate(X_test, y_test))

188/188 [==============================] - 9s 46ms/step - loss: 0.5836 - categorical_accuracy: 0.8703 - f1_score: 0.8650
[0.583574652671814, 0.8702837824821472, 0.8649557828903198]
27/27 [==============================] - 1s 46ms/step - loss: 1.9962 - categorical_accuracy: 0.4875 - f1_score: 0.4543
[1.9961954355239868, 0.4874851107597351, 0.4542781710624695]
181/181 [==============================] - 8s 46ms/step - loss: 2.0209 - categorical_accuracy: 0.4789 - f1_score: 0.4756
[2.020860195159912, 0.47892919182777405, 0.47560492157936096]


In [17]:
hist = model.fit_generator(datagen.flow(X_tn, y_tn, batch_size=16), epochs=800, validation_data=(X_val, y_val), callbacks=[WandbCallback()])

Epoch 1/800


/tmp/ipykernel_363/1117950546.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(datagen.flow(X_tn, y_tn, batch_size=16), epochs=800, validation_data=(X_val, y_val), callbacks=[WandbCallback()])


322/322 [==============================] - 38s 119ms/step - loss: 1.3227 - categorical_accuracy: 0.6238 - f1_score: 0.6163 - val_loss: 2.0655 - val_categorical_accuracy: 0.4827 - val_f1_score: 0.4444
Epoch 2/800
322/322 [==============================] - 38s 119ms/step - loss: 1.3046 - categorical_accuracy: 0.6265 - f1_score: 0.6197 - val_loss: 2.0611 - val_categorical_accuracy: 0.4720 - val_f1_score: 0.4348
Epoch 3/800
322/322 [==============================] - 38s 119ms/step - loss: 1.3388 - categorical_accuracy: 0.6230 - f1_score: 0.6159 - val_loss: 2.0065 - val_categorical_accuracy: 0.4863 - val_f1_score: 0.4496
Epoch 4/800
322/322 [==============================] - 39s 120ms/step - loss: 1.3025 - categorical_accuracy: 0.6276 - f1_score: 0.6200 - val_loss: 2.0261 - val_categorical_accuracy: 0.4756 - val_f1_score: 0.4360
Epoch 5/800
322/322 [==============================] - 39s 121ms/step - loss: 1.3203 - categorical_accuracy: 0.6220 - f1_score: 0.6159 - val_loss: 2.0324 - val_cate

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



322/322 [==============================] - 38s 118ms/step - loss: 1.1716 - categorical_accuracy: 0.6655 - f1_score: 0.6582 - val_loss: 2.0101 - val_categorical_accuracy: 0.4851 - val_f1_score: 0.4472
Epoch 54/800
322/322 [==============================] - 38s 118ms/step - loss: 1.1726 - categorical_accuracy: 0.6663 - f1_score: 0.6586 - val_loss: 2.0477 - val_categorical_accuracy: 0.4911 - val_f1_score: 0.4510
Epoch 55/800
322/322 [==============================] - 38s 118ms/step - loss: 1.1910 - categorical_accuracy: 0.6597 - f1_score: 0.6529 - val_loss: 2.0441 - val_categorical_accuracy: 0.4779 - val_f1_score: 0.4412
Epoch 56/800
322/322 [==============================] - 39s 119ms/step - loss: 1.2008 - categorical_accuracy: 0.6531 - f1_score: 0.6457 - val_loss: 2.0075 - val_categorical_accuracy: 0.4863 - val_f1_score: 0.4509
Epoch 57/800
322/322 [==============================] - 38s 118ms/step - loss: 1.1693 - categorical_accuracy: 0.6585 - f1_score: 0.6510 - val_loss: 2.0298 - val_

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



322/322 [==============================] - 39s 120ms/step - loss: 0.5740 - categorical_accuracy: 0.8284 - f1_score: 0.8223 - val_loss: 2.1147 - val_categorical_accuracy: 0.5149 - val_f1_score: 0.4715
Epoch 710/800
322/322 [==============================] - 39s 120ms/step - loss: 0.5328 - categorical_accuracy: 0.8457 - f1_score: 0.8414 - val_loss: 2.1365 - val_categorical_accuracy: 0.5161 - val_f1_score: 0.4718
Epoch 714/800
322/322 [==============================] - 39s 121ms/step - loss: 0.5612 - categorical_accuracy: 0.8422 - f1_score: 0.8369 - val_loss: 2.1135 - val_categorical_accuracy: 0.5113 - val_f1_score: 0.4699
Epoch 715/800
322/322 [==============================] - 38s 119ms/step - loss: 0.5810 - categorical_accuracy: 0.8276 - f1_score: 0.8215 - val_loss: 2.1054 - val_categorical_accuracy: 0.5173 - val_f1_score: 0.4758
Epoch 716/800
322/322 [==============================] - 38s 118ms/step - loss: 0.5606 - categorical_accuracy: 0.8346 - f1_score: 0.8294 - val_loss: 2.0958 - 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



322/322 [==============================] - 38s 118ms/step - loss: 0.5373 - categorical_accuracy: 0.8459 - f1_score: 0.8408 - val_loss: 2.0837 - val_categorical_accuracy: 0.5030 - val_f1_score: 0.4653
Epoch 762/800
322/322 [==============================] - 38s 119ms/step - loss: 0.5257 - categorical_accuracy: 0.8519 - f1_score: 0.8467 - val_loss: 2.1112 - val_categorical_accuracy: 0.5197 - val_f1_score: 0.4817
Epoch 766/800
322/322 [==============================] - 38s 118ms/step - loss: 0.5269 - categorical_accuracy: 0.8492 - f1_score: 0.8447 - val_loss: 2.0961 - val_categorical_accuracy: 0.5101 - val_f1_score: 0.4675
Epoch 767/800
322/322 [==============================] - 39s 120ms/step - loss: 0.5277 - categorical_accuracy: 0.8511 - f1_score: 0.8460 - val_loss: 2.0851 - val_categorical_accuracy: 0.5054 - val_f1_score: 0.4630
Epoch 768/800
322/322 [==============================] - 38s 117ms/step - loss: 0.5296 - categorical_accuracy: 0.8478 - f1_score: 0.8428 - val_loss: 2.1211 - 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [18]:
print(model.evaluate(X_train, y_train))
print(model.evaluate(X_val, y_val))
print(model.evaluate(X_test, y_test))

188/188 [==============================] - 9s 46ms/step - loss: 0.3247 - categorical_accuracy: 0.9282 - f1_score: 0.9235
[0.3247394859790802, 0.9282137155532837, 0.9234686493873596]
27/27 [==============================] - 1s 45ms/step - loss: 2.0735 - categorical_accuracy: 0.5137 - f1_score: 0.4740
[2.0735132694244385, 0.5137068033218384, 0.47401610016822815]
181/181 [==============================] - 8s 46ms/step - loss: 2.1639 - categorical_accuracy: 0.5147 - f1_score: 0.5115
[2.1638591289520264, 0.5146805047988892, 0.5114830136299133]


In [ ]:
wandb.finish()